# View Catalog Demo (REQ_047)

Demonstrates the `EpochContext` pattern — pin a training moment once,
then examine any view through that same lens without re-specifying the epoch.

All views are accessed through `variant.at(epoch).view(name)` or the
`variant.view(name)` convenience shortcut. No direct imports of
`ArtifactLoader`, renderer functions, or `export_variant_visualization()`.

In [ ]:
from miscope import catalog, load_family

# Inspect all available views
print(f"{len(catalog.names())} views registered:")
for name in catalog.names():
    print(f"  {name}")

In [ ]:
family = load_family("modulo_addition_1layer")
variant = family.get_variant(prime=113, seed=999)
print(variant)

## Shared Epoch Cursor

Pin a single training moment. Every view accessed through this context
reflects the same epoch — the cursor is set once, not per-view.

In [ ]:
# Choose an epoch near grokking onset
EPOCH = 10000
ctx = variant.at(epoch=EPOCH)

In [ ]:
# Data source 1: metadata.json (no artifact loader involved)
ctx.view("loss_curve").show()

In [ ]:
# Data source 2: per-epoch artifact (.npz snapshot)
ctx.view("dominant_frequencies").show()

In [ ]:
# Data source 3: cross-epoch artifact (cursor highlights EPOCH on trajectory)
ctx.view("parameter_trajectory").show()

## Convenience Shortcut

`variant.view(name)` is sugar for `variant.at(epoch=None).view(name)`.
Per-epoch views resolve to the first available artifact epoch automatically.

In [ ]:
variant.view("loss_curve").show()

In [ ]:
# First available artifact epoch (not necessarily 0)
variant.view("dominant_frequencies").show()

## Export to File

`BoundView.export(format, path)` writes a static file without any
direct interaction with `export_variant_visualization()`.

In [ ]:
from pathlib import Path

out = Path("exports")
out.mkdir(exist_ok=True)

path = ctx.view("loss_curve").export("html", out / "loss_curve.html")
print(f"Exported to {path}")

## Accessing Multiple Views at the Same Epoch

Compare onset of structure across different lenses at a single
pinned training moment.

In [ ]:
for name in ["loss_curve", "dominant_frequencies", "parameter_trajectory", "specialization_trajectory"]:
    print(f"--- {name} ---")
    ctx.view(name).show()

## Raw Figure Access

`BoundView.figure()` returns the Plotly `Figure` object for use in
dashboards or custom layouts.

In [ ]:
fig = ctx.view("loss_curve").figure()
print(type(fig))  # plotly.graph_objects.Figure
print(f"Traces: {len(fig.data)}")